### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [3]:
user_movie_subset.head(2)

movie_id,73486,75314,68646,99685
user_id,,,,
265,10.0,10.0,10.0,10.0
1023,10.0,4.0,9.0,10.0


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [26]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    # num_ratings = n_users*n_movies # total number of ratings in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) 
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
        # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [9]:
ratings_mat[0, 1]

10.0

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [27]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.266980
2 		 18.014661
3 		 4.689247
4 		 2.838501
5 		 2.674800
6 		 2.621332
7 		 2.577205
8 		 2.532069
9 		 2.483298
10 		 2.429681


In [28]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.2641903    8.61283183  10.67276073  10.12943556]
 [  9.03961562   7.09883552   8.95756593   8.87116716]
 [  7.8270712    7.04720377   8.04409892   7.88347326]
 [  9.30713744   8.8593369    9.7673867    9.62905812]
 [  8.79150809   6.78091004   9.37280247   8.41443424]
 [  7.04461197   5.55588614   7.26211991   6.9680294 ]
 [  8.93778903   8.44188623   9.72752348   9.08760379]
 [  8.04898136   6.97816231   8.90509785   8.1614512 ]
 [  8.37679644   7.95523313   8.6154866    8.51512621]
 [  8.0071234    6.54477367   8.13935341   7.50374767]
 [  8.91180662   7.57376344   9.34305955   8.92707628]
 [  9.51995586   8.64503225   9.96061005   9.54780826]
 [  9.42796436   8.19348461   9.61562763   9.18850049]
 [  6.41820556   5.99399404   6.76104944   6.80521855]
 [  9.84744527   8.3505271   10.31810181   9.69430911]
 [  9.66160848   8.65264851  10.22619506   9.71350434]
 [  8.33306847   7.50857847   8.44878201   8.32980392]
 [  7.20790227   6.31306634   7.00149523   7.32421137]
 [  8.0976

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [29]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 48.537440
2 		 20.335028
3 		 5.661381
4 		 3.003446
5 		 2.692305
6 		 2.619161
7 		 2.572746
8 		 2.526060
9 		 2.473776
10 		 2.414334
11 		 2.347053
12 		 2.271650
13 		 2.188229
14 		 2.097342
15 		 2.000035
16 		 1.897838
17 		 1.792683
18 		 1.686739
19 		 1.582201
20 		 1.481043
21 		 1.384821
22 		 1.294550
23 		 1.210690
24 		 1.133221
25 		 1.061795
26 		 0.995899
27 		 0.934995
28 		 0.878623
29 		 0.826443
30 		 0.778234
31 		 0.733863
32 		 0.693239
33 		 0.656268
34 		 0.622818
35 		 0.592697
36 		 0.565651
37 		 0.541365
38 		 0.519482
39 		 0.499623
40 		 0.481401
41 		 0.464442
42 		 0.448399
43 		 0.432957
44 		 0.417841
45 		 0.402817
46 		 0.387697
47 		 0.372333
48 		 0.356619
49 		 0.340489
50 		 0.323915
51 		 0.306905
52 		 0.289501
53 		 0.271775
54 		 0.253827
55 		 0.235780
56 		 0.217774
57 		 0.199963
58 		 0.182503
59 		 0.165550
60 		 0.149249
61 		 0.133732
62 		 0.119110
63 		 0.105468
64 		

In [30]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000205  10.00000109  10.00000065   9.99999776]
 [  9.99999838   3.99999917   8.99999949  10.00000171]
 [  8.00000074   9.00000042  10.00000024   4.99999915]
 [  9.00000054   8.00000029  10.00000016   9.99999939]
 [ 10.00000379   5.00000195   9.00000116   8.9999959 ]
 [  6.00000042   4.00000018  10.00000008   5.99999951]
 [  8.99999808   7.99999892   9.99999933   9.00000205]
 [  9.99999981   4.99999986   8.9999999    8.00000019]
 [  7.00000237   8.00000118  10.00000069   7.99999744]
 [  9.00000035   5.00000014   9.00000005   6.99999959]
 [  8.99999939   7.99999963   9.99999975   8.00000062]
 [  9.0000026   10.00000129  10.00000073   8.99999716]
 [  9.9999998    8.99999981   9.99999983   8.00000017]
 [  4.99999873   7.99999926   4.99999952   8.00000137]
 [  9.9999988    7.99999932   9.99999956  10.00000126]
 [  8.99999951   8.9999997    9.9999998   10.0000005 ]
 [  8.99999351   7.99999666   7.999998     8.00000694]
 [ 10.00000157   8.00000086   1.00000055   9.99999834]
 [  4.9999

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [31]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [32]:
# run SVD on the matrix with the missing value
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 40.757288
2 		 13.935748
3 		 4.307928
4 		 2.920402
5 		 2.685844
6 		 2.595908
7 		 2.532149
8 		 2.470467
9 		 2.404521
10 		 2.332476
11 		 2.254001
12 		 2.169564
13 		 2.080268
14 		 1.987762
15 		 1.894126
16 		 1.801672
17 		 1.712694
18 		 1.629189
19 		 1.552628
20 		 1.483804
21 		 1.422808
22 		 1.369108
23 		 1.321715
24 		 1.279363
25 		 1.240695
26 		 1.204391
27 		 1.169265
28 		 1.134306
29 		 1.098698
30 		 1.061817
31 		 1.023223
32 		 0.982643
33 		 0.939965
34 		 0.895227
35 		 0.848604
36 		 0.800398
37 		 0.751019
38 		 0.700966
39 		 0.650792
40 		 0.601078
41 		 0.552394
42 		 0.505267
43 		 0.460153
44 		 0.417415
45 		 0.377313
46 		 0.340006
47 		 0.305556
48 		 0.273944
49 		 0.245090
50 		 0.218864
51 		 0.195111
52 		 0.173661
53 		 0.154338
54 		 0.136968
55 		 0.121388
56 		 0.107441
57 		 0.094981
58 		 0.083871
59 		 0.073987
60 		 0.065210
61 		 0.057432
62 		 0.050552
63 		 0.044476
64 		

In [33]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.923891378892312:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [34]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 39.143128
2 		 12.719889
3 		 4.117296
4 		 2.942359
5 		 2.747075
6 		 2.677786
7 		 2.633286
8 		 2.592433
9 		 2.549351
10 		 2.501901
11 		 2.448946
12 		 2.389665
13 		 2.323389
14 		 2.249604
15 		 2.167989
16 		 2.078475
17 		 1.981316
18 		 1.877145
19 		 1.767023
20 		 1.652450


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [41]:
print(first_1000_users.shape)
first_1000_users[0]

(1000, 31245)


matrix([[ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [38]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [39]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
